In [1]:
# Fourth meeting on sherlock
# Notebook to assist reannnotating the sherlock benchmark dataset with some additional notes about the dataset

In [1]:
from datetime import datetime
from os.path import join
from os import listdir
from os import makedirs
import re #for camel case conversion
from collections import Counter

import numpy as np
import pandas as pd
from pyarrow.parquet import ParquetFile

from sherlock import helpers

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
# from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
#     convert_string_lists_to_lists,
    prepare_feature_extraction,
#     load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

### Utils

In [2]:
def camel_case(s):
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
    s = ''.join([s[0].lower(), s[1:]])
    return s

In [3]:
def sherlock_case(s):
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
    s = ''.join([s[0].lower(), s[1:]])
    s = ''.join(map(lambda x: x if x.islower() else " "+x, s))
    return s

## Loading the annotation data
##### Make sure to unzip tables.zip in the /data/data/gittables_benchmark directory before proceeding

In [4]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:04.060688 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.442340 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.139904 seconds.


[nltk_data] Downloading package punkt to /home/senn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/senn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

W0523 10:44:11.208704 140592511874880 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0523 10:44:11.209521 140592511874880 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0523 10:44:11.211674 140592511874880 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Varia

### Load Gittables

In [6]:
path = '../data/data/gittables_benchmark'
path_removed = '../data/data/gittables_benchmark/non_reannotated/removed/'
types = np.load(f"../model_files/classes_sherlock.npy", allow_pickle=True)
# print(types)
# with open(join(path, '../../types.json'), 'r') as f:  
#         types = json.load(f)['type78']

dbpedia_gt = pd.read_csv(join(path, 'dbpedia_gt.csv'))
schema_gt = pd.read_csv(join(path, 'schema_gt.csv'))
dbpedia_labels = pd.read_csv(join(path, 'dbpedia_labels.csv'))
dbpedia_targets = pd.read_csv(join(path, 'dbpedia_targets.csv'))
# test_table = pd.read_csv(join(path, 'tables/GitTables_1501.csv'), dtype='object')

In [7]:
path_out = '../data/data/gittables_benchmark/reannotated_temp'
makedirs(path_out, exist_ok=True)
columns = []
col_true_types = []
table_review_needed = {}
table_predicted_types = {}
table_true_types = {}
table_ids = []
col_ids = []
table_col_amounts = []
filepaths = sorted([join(path, 'tables/', f) for f in listdir(join(path, 'tables/')) if f.endswith('.csv')])

#Go over the tables in the dataset
for fp in filepaths:
    table_id = fp[fp.rfind('/')+1:-4][:14]
    table_df = pd.read_csv(fp)
#     table_df_no_na = pd.read_csv(fp, dtype=str, na_filter=False)
#     table_df_no_na.convert_dtypes()
#     table_df.convert_dtypes()
    dbpedia_table_gts = dbpedia_gt[dbpedia_gt['table_id'].str.contains(table_id)]
    dbpedia_table_gts = dbpedia_table_gts[['table_id', 'target_column', 'annotation_label']]
    dbpedia_table_gts['table_id'] = dbpedia_table_gts['table_id'].apply(lambda x: table_id)
    schema_table_gts = schema_gt[schema_gt['table_id'].str.contains(table_id)]
    schema_table_gts = schema_table_gts[['table_id', 'target_column', 'annotation_label']]
    schema_table_gts['table_id'] = schema_table_gts['table_id'].apply(lambda x: table_id)
    table_gts = pd.merge(dbpedia_table_gts, schema_table_gts, how='outer')
#     print(table_gts)

    columns = []
    col_true_types = []
    
    for index, gt_row in table_gts.iterrows():
        col_type = sherlock_case(gt_row['annotation_label'])
        # We only want types that are used in sherlock
        if col_type not in types:
            continue
        col_name = 'col'+ str(gt_row['target_column'])
        col_id = table_id[table_id.rfind('_')+1:] + str(gt_row['target_column'])
        col = table_df[col_name].convert_dtypes()
#         col_no_na = table_df_no_na[col_name]
        
        if pd.isnull(col).sum() > 0: #Base f1 = 0.31245450119322055
            #count returns the amount of valid values in the column, if column does
            #not contain any valid values, there is no point in trying to predict/use the column
            if col.count() == 0: # 0.35952832125142853
                continue
#             if col_type in ('year'):
#                 print(f'col of type {col_type} contains invalid values, file name: {fp}, col: {col_name}')
#                 print(col.astype(object).fillna(''))
                
        columns.append(list(map(str, col.astype(object).fillna('').to_list()))) #columns need to be converted to list of strings for preprocessing later on
        col_true_types.append(col_type)
        col_ids.append(col_id)
    if(len(columns)==0):
        continue
    df = pd.DataFrame(zip(*columns), columns=col_true_types)
    table_col_amounts.append(len(df.columns))
    data = pd.Series(
    [
        *columns #expand/unpack the list
    ],
    name="values"
    )
    extract_features(
    join(path_out, "temporary.csv"),
    data
    )
    feature_vectors = pd.read_csv(join(path_out, "temporary.csv"), dtype=np.float32)
    p_types = model.predict(feature_vectors, "sherlock").tolist()
    table_predicted_types[table_id] = p_types
    table_true_types[table_id] = col_true_types
    table_review_needed[table_id] = not(p_types == col_true_types)
#     df.to_csv(path_out+'/'+table_id+'.csv') #DO NOT UNCOMMENT, unless you want to recreate non-reannotated dataset
    table_ids.append(table_id)
col_amount_freq = Counter(table_col_amounts)
col_amount_freq

Extracting Features: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 148.54it/s]


Exporting 1588 column features
Exporting 1588 column features


Extracting Features: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 112.10it/s]


Exporting 1588 column features
Exporting 1588 column features
Exporting 1588 column features


Extracting Features: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 43.12it/s]


...

Extracting Features: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 202.51it/s]


Exporting 1588 column features
Exporting 1588 column features
Exporting 1588 column features


Extracting Features: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 241.52it/s]


Exporting 1588 column features
Exporting 1588 column features


Counter({1: 299, 2: 68, 4: 32, 3: 80, 5: 2})

In [8]:
# columns per table distribution
col_amount_freq_dict = dict(col_amount_freq)
df_dict = {'col_amount': list(col_amount_freq_dict.keys()), 'freq': list(col_amount_freq_dict.values())}
print(df_dict)
col_amount_freq_df = pd.DataFrame(df_dict)
alt.Chart(col_amount_freq_df.reset_index()).mark_bar(size=40).encode(
    x = alt.X('col_amount',
              type='ordinal',
              title = 'Amount of Columns',
#               sort=alt.EncodingSortField(
#                 field="values",  
#                 order="descending"),
              ),
    y = alt.Y('freq', title='Number of Tables'),
).configure_axis(
    labelFontSize=18,
    titleFontSize=18
).configure_axisX(
    labelAngle=0
).properties(width=500,height=200)

{'col_amount': [1, 2, 4, 3, 5], 'freq': [299, 68, 32, 80, 2]}


alt.Chart(...)

In [10]:
# Take a closer look at true type distribution
true_types = [_type for typelist in table_true_types.values() for _type in typelist]
print(len(true_types)) #amount of columns in benchmark
type_frequency_dict = dict(Counter(true_types))
print(type_frequency_dict)

df_dict = {'type': list(type_frequency_dict.keys()), 'freq': list(type_frequency_dict.values())}
print(df_dict)
type_frequency_df = pd.DataFrame(df_dict)
alt.Chart(type_frequency_df.reset_index()).mark_bar(size=20).encode(
    x = alt.X('type',
              type='ordinal',
              title = 'Type',
              sort=alt.EncodingSortField(
                field="freq",  
                order="descending"),
              ),
    y = alt.Y('freq', title='Frequency'),
).configure_axis(
    labelFontSize=18,
    titleFontSize=18
).properties(width=800,height=200)
# temp_df = pd.DataFrame(true_types, columns=['type'])
# # type_occurrence_df = type_occurrence_df.groupby('type').count()
# type_occurrence_df = pd.DataFrame(set(true_types), columns=['type'])
# type_occurrence_df['freq_count'] = temp_df.groupby('type')['type'].transform('count')
# # print(type_occurrence_df)
# alt.Chart(type_occurrence_df).mark_bar(size=20).encode(
#     x = alt.X('type:O',
#               title = 'Semantic Types',
#               sort=alt.EncodingSortField(
#                 field="freq_count",  
#                 order="descending"),
#              ),
#     y = alt.Y('freq_count', title='Number of Samples'),
# #     color=alt.condition(
# #         alt.Predicate(alt.FieldOneOfPredicate(field='type', oneOf=shown_types)),
# #         alt.value('orange'),
# #         alt.value('steelblue')
# #     ),
# ).configure_axis(
#     labelFontSize=18,
#     titleFontSize=18
# ).properties(width=800,height=200)

813
{'type': 144, 'state': 20, 'description': 35, 'year': 102, 'class': 64, 'status': 15, 'depth': 13, 'notes': 6, 'code': 17, 'classification': 1, 'location': 2, 'range': 1, 'gender': 1, 'component': 3, 'product': 3, 'category': 5, 'company': 2, 'manufacturer': 2, 'city': 5, 'country': 4, 'name': 210, 'duration': 11, 'language': 4, 'rank': 100, 'species': 27, 'address': 1, 'county': 1, 'team': 2, 'family': 2, 'weight': 3, 'capacity': 1, 'region': 1, 'operator': 1, 'origin': 1, 'collection': 1, 'order': 1, 'age': 1}
{'type': ['type', 'state', 'description', 'year', 'class', 'status', 'depth', 'notes', 'code', 'classification', 'location', 'range', 'gender', 'component', 'product', 'category', 'company', 'manufacturer', 'city', 'country', 'name', 'duration', 'language', 'rank', 'species', 'address', 'county', 'team', 'family', 'weight', 'capacity', 'region', 'operator', 'origin', 'collection', 'order', 'age'], 'freq': [144, 20, 35, 102, 64, 15, 13, 6, 17, 1, 2, 1, 1, 3, 3, 5, 2, 2, 5, 4

alt.Chart(...)

In [90]:
# frequency of equally type tables
frequently_occurring_types_1 = ['state', 'type']
frequently_occurring_types_2 = ['name', 'rank', 'year']
occurrence_count = 0
for table_tt in table_true_types.values():
    if set(frequently_occurring_types_2).issubset(table_tt):
        occurrence_count += 1
print(occurrence_count)       

96


In [9]:
table_id = 'GitTables_1733'
print(table_review_needed[table_id])
print(table_predicted_types[table_id])

True
['sex']


In [10]:
# manually comparing true types to predicted types
for table_id in table_ids:
    if table_review_needed[table_id]:
        print(table_id)
        print(table_true_types[table_id])
        print(table_predicted_types[table_id])

GitTables_1503
['state', 'type']
['type', 'location']
GitTables_1504
['description']
['notes']
GitTables_1511
['state', 'type']
['category', 'type']
GitTables_1512
['type']
['origin']
GitTables_1516
['class']
['address']
GitTables_1524
['type']
['symbol']
GitTables_1525
['class']
['order']
GitTables_1527
['status']
['artist']
GitTables_1529
['class']
['address']
GitTables_1531
['type']
['address']
GitTables_1533
['depth']
['address']
GitTables_1536
['notes']
['description']
GitTables_1537
['class']
['address']
GitTables_1538
['class', 'code']
['sex', 'rank']
GitTables_1539
['depth']
['address']
GitTables_1542
['type']
['address']
GitTables_1544
['class', 'description']
['address', 'command']
GitTables_1545
['type']
['address']
GitTables_1546
['state', 'type']
['category', 'type']
GitTables_1547
['class']
['address']
GitTables_1548
['type']
['format']
GitTables_1553
['class']
['address']
GitTables_1558
['type']
['code']
GitTables_1560
['state', 'type']
['category', 'location']
GitTables

['code', 'name']
['command', 'component']
GitTables_2496
['name', 'rank', 'species', 'year']
['species', 'species', 'species', 'birth Date']
GitTables_2497
['class']
['product']
GitTables_2503
['name', 'rank', 'year']
['species', 'species', 'year']
GitTables_2508
['origin', 'type']
['address', 'type']
GitTables_2511
['name', 'rank', 'species', 'year']
['species', 'species', 'species', 'year']
GitTables_2513
['notes']
['description']
GitTables_2516
['name']
['position']
GitTables_2518
['name']
['notes']
GitTables_2521
['category', 'name']
['category', 'description']
GitTables_2525
['name']
['address']
GitTables_2528
['name']
['address']
GitTables_2533
['collection', 'status', 'type']
['state', 'rank', 'rank']
GitTables_2539
['name', 'rank', 'year']
['species', 'species', 'year']
GitTables_2542
['name', 'rank', 'year']
['species', 'species', 'year']
GitTables_2543
['name', 'rank', 'year']
['species', 'species', 'birth Date']
GitTables_2550
['name']
['component']
GitTables_2551
['name', '

In [65]:
# take a closer look at the types present in the benchmark
included_types = list(set(types) & set(true_types))
print(len(included_types))

37


In [12]:
print(len(table_col_amounts))

481


In [13]:
# GitTables_2027, GitTables_2065, GitTables_2212(removed), GitTables_2213 (reannot.),
# Gittables_1755 -> removed last 2 cols
# GitTables_2230, GITTables_2231(removed), GitTables_2425, GITTables_2533(removed), 
# GitTables_2575(reannot.), GitTables_2945, GitTables_2668
# GitTables_2002 (removed)
# GitTables_2158, GitTables_2396, multiple ground thruths: ['code', 'name'] -> ['code', 'state'] or ['state', 'state'], depending on table context or column context
# GitTables_2232: same as prev ['code', 'name'] -> ?
# table with ['state', 'type'], is in the dataset many times, 
# same for 1/2 column tables with a class or type from a programming language, make up around 70% of the dataset
# same for 1 col tables with random numeric values like depth or duration (GitTables_2022, GitTables_1937)
# Surprisingly well: GitTables_1918

In [14]:
# GitTables_2004, GitTables_2029 (weird address prediction)
# these are tables with more than 3 cols which also appear freq in dataset, 
# we also saw this last week so I decided to do some research.
# https://en.wikipedia.org/wiki/Taxonomic_rank#:~:text=There%20are%20seven%20main%20taxonomic,a%20synonym%20for%20dominion%20(lat., https://www.irmng.org/aphia.php?p=taxdetails&id=1041282, https://www.irmng.org/aphia.php?p=taxdetails&id=10255148
# This table appears in the dataset a lot of times, give or take the same, sometimes the species or status column is left out
# this gives us a distorted picture of the prediction results -> name is predicted falsly a lot because of reannotation to species
# and year is predicted correctly many times because it appears frequently, maybe merge into 1 table, but then previous results would be 
# 'useless', and we would have a lot less test data.

In [15]:
# GitTables_2059, one of the interesting tables with 5 cols, county vs. city (= provincie vs. stad)

In [16]:
# GitTables_2302,GitTables_2251, GitTables_2634, GitTables_2430, GitTables_2745: non-english col values

In [17]:
# description/class/type/name/address/notes (punctuation confusing sherlock)
# while reannotating hard to pick between class/type/classification/category or team/team name (but, not frequently in this dataset)

In [95]:
# looking at removed tables
filepaths = listdir(path_removed)
for fp in filepaths:
    df = pd.read_csv(join(path_removed, fp))
    print(df)

    Unnamed: 0 language       type
0            0        *        NaN
1            1        *  component
2            2        *  component
3            3        *  component
4            4        *  component
5            5        *  component
6            6        *  component
7            7        *  component
8            8        *  component
9            9        *  component
10          10        *  component
11          11        *  component
12          12        *  component
13          13        *  component
14          14        *  component
15          15        *  component
16          16        *  component
17          17        *  component
18          18        *  component
19          19        *  component
20          20        *  component
    Unnamed: 0  collection  status  type
0            0         0.0     0.0   0.0
1            1         0.0     0.0   0.0
2            2         0.0     0.0   0.0
3            3         0.0     0.0   0.0
4            4         0.